In [1]:
import pandas as pd
import requests

## Overall Feed

In [2]:
r = requests.get("https://gbfs.citibikenyc.com/gbfs/2.3/gbfs.json")

In [3]:
feed_data = r.json()

In [4]:
feed_data

{'data': {'en': {'feeds': [{'url': 'https://gbfs.lyft.com/gbfs/2.3/bkn/gbfs.json',
     'name': 'gbfs'},
    {'url': 'https://gbfs.lyft.com/gbfs/2.3/bkn/en/system_information.json',
     'name': 'system_information'},
    {'url': 'https://gbfs.lyft.com/gbfs/2.3/bkn/en/station_information.json',
     'name': 'station_information'},
    {'url': 'https://gbfs.lyft.com/gbfs/2.3/bkn/en/station_status.json',
     'name': 'station_status'},
    {'url': 'https://gbfs.lyft.com/gbfs/2.3/bkn/en/free_bike_status.json',
     'name': 'free_bike_status'},
    {'url': 'https://gbfs.lyft.com/gbfs/2.3/bkn/en/system_hours.json',
     'name': 'system_hours'},
    {'url': 'https://gbfs.lyft.com/gbfs/2.3/bkn/en/system_calendar.json',
     'name': 'system_calendar'},
    {'url': 'https://gbfs.lyft.com/gbfs/2.3/bkn/en/system_regions.json',
     'name': 'system_regions'},
    {'url': 'https://gbfs.lyft.com/gbfs/2.3/bkn/en/system_pricing_plans.json',
     'name': 'system_pricing_plans'},
    {'url': 'https://gb

## Station Status

In [5]:
station_r = requests.get("https://gbfs.lyft.com/gbfs/2.3/bkn/en/station_status.json")

In [6]:
station_df = pd.DataFrame.from_dict(station_r.json().get('data').get('stations'))

In [7]:
station_df.head()

,station_id,num_docks_available,is_installed,vehicle_types_available,num_ebikes_available,num_bikes_disabled,num_bikes_available,is_returning,num_docks_disabled,last_reported,is_renting,num_scooters_available,num_scooters_unavailable
0,0bd9bd58-42e6-4680-9d19-83943372221f,0,0,"[{'count': 0, 'vehicle_type_id': '1'}, {'count...",0,0,0,0,0,1729514620,0,NaN,NaN
1,cc5f0e68-dd5f-4db1-81c7-04a4d6674fba,6,1,"[{'count': 10, 'vehicle_type_id': '1'}, {'coun...",1,2,11,1,0,1730254749,1,0.0,0.0
2,1799624430022654108,4,1,"[{'count': 15, 'vehicle_type_id': '1'}, {'coun...",2,0,17,1,0,1730254747,1,0.0,0.0
3,66de2199-0aca-11e7-82f6-3863bb44ef7c,1,1,"[{'count': 26, 'vehicle_type_id': '1'}, {'coun...",1,2,27,1,0,1730254752,1,0.0,0.0
4,b4a320d6-2611-4940-96ca-17fb297ec826,6,1,"[{'count': 9, 'vehicle_type_id': '1'}, {'count...",10,0,19,1,0,1730254754,1,0.0,0.0


In [8]:
station_df["vehicle_types_available"][1]

[{'count': 10, 'vehicle_type_id': '1'}, {'count': 1, 'vehicle_type_id': '2'}]

### Side Note

What the heck do those `vehicle_type_id`s even mean? Look here: https://gbfs.lyft.com/gbfs/2.3/bkn/es/vehicle_types.json

In [9]:
r_vehicle_types = requests.get("https://gbfs.lyft.com/gbfs/2.3/bkn/es/vehicle_types.json")

In [10]:
r_vehicle_types.json()

{'data': {'vehicle_types': [{'propulsion_type': 'human',
    'form_factor': 'bicycle',
    'vehicle_type_id': '1'},
   {'propulsion_type': 'electric_assist',
    'form_factor': 'bicycle',
    'vehicle_type_id': '2',
    'max_range_meters': 54717.56}]},
 'last_updated': 1730254884,
 'ttl': 60,
 'version': '2.3'}

In [11]:
# filter the list of vehicle types available for the given id
def filter_by_vehicle_type(count_list, target_id):
    return next((item["count"] for item in count_list if item["vehicle_type_id"] == str(target_id)), 0)

In [12]:
station_df["electric_bikes_available"] = station_df["vehicle_types_available"].apply(lambda x: filter_by_vehicle_type(x, 2))
station_df["regular_bikes_available"] = station_df["vehicle_types_available"].apply(lambda x: filter_by_vehicle_type(x, 1))

In [13]:
station_df

,station_id,num_docks_available,is_installed,vehicle_types_available,num_ebikes_available,num_bikes_disabled,num_bikes_available,is_returning,num_docks_disabled,last_reported,is_renting,num_scooters_available,num_scooters_unavailable,electric_bikes_available,regular_bikes_available
0,0bd9bd58-42e6-4680-9d19-83943372221f,0,0,"[{'count': 0, 'vehicle_type_id': '1'}, {'count...",0,0,0,0,0,1729514620,0,NaN,NaN,0,0
1,cc5f0e68-dd5f-4db1-81c7-04a4d6674fba,6,1,"[{'count': 10, 'vehicle_type_id': '1'}, {'coun...",1,2,11,1,0,1730254749,1,0.0,0.0,1,10
2,1799624430022654108,4,1,"[{'count': 15, 'vehicle_type_id': '1'}, {'coun...",2,0,17,1,0,1730254747,1,0.0,0.0,2,15
3,66de2199-0aca-11e7-82f6-3863bb44ef7c,1,1,"[{'count': 26, 'vehicle_type_id': '1'}, {'coun...",1,2,27,1,0,1730254752,1,0.0,0.0,1,26
4,b4a320d6-2611-4940-96ca-17fb297ec826,6,1,"[{'count': 9, 'vehicle_type_id': '1'}, {'count...",10,0,19,1,0,1730254754,1,0.0,0.0,10,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2222,66dd5a42-0aca-11e7-82f6-3863bb44ef7c,7,1,"[{'count': 11, 'vehicle_type_id': '1'}, {'coun...",3,1,14,1,0,1730254794,1,0.0,0.0,3,11
2223,66dddd28-0aca-11e7-82f6-3863bb44ef7c,6,1,"[{'count': 1, 'vehicle_type_id': '1'}, {'count...",7,0,8,1,0,1730254817,1,0.0,0.0,7,1
2224,66ddd93e-0aca-11e7-82f6-3863bb44ef7c,0,1,"[{'count': 5, 'vehicle_type_id': '1'}, {'count...",8,1,13,1,0,1730254824,1,0.0,0.0,8,5
2225,46813ecf-8df4-4c8f-9579-0179e0b36ba6,7,1,"[{'count': 7, 'vehicle_type_id': '1'}, {'count...",2,2,9,1,0,1730254826,1,0.0,0.0,2,7


## Station Information

Alright, now we know about the stations, but we need some metadata

In [14]:
r_station_information = requests.get("https://gbfs.lyft.com/gbfs/2.3/bkn/es/station_information.json")

In [15]:
station_metadata = pd.DataFrame.from_dict(r_station_information.json().get('data').get('stations'))

## Tie It All Together

In [16]:
all_station_data = pd.merge(
    station_metadata,
    station_df,
    how="left",
    on="station_id"
)

In [17]:
all_station_data.sort_values("electric_bikes_available", ascending=False)[["station_id", "name", "electric_bikes_available", "capacity"]]

,station_id,name,electric_bikes_available,capacity
865,66dc3f08-0aca-11e7-82f6-3863bb44ef7c,W 41 St & 8 Ave,76,105
719,66dc7de9-0aca-11e7-82f6-3863bb44ef7c,W 43 St & 10 Ave,60,117
1235,2c321ea3-e1dd-4d32-be70-bd158fe1f568,W 44 St & 11 Ave,54,79
722,66dd51e6-0aca-11e7-82f6-3863bb44ef7c,Riverside Blvd & W 67 St,54,79
1906,66dc7d58-0aca-11e7-82f6-3863bb44ef7c,12 Ave & W 40 St,47,76
...,...,...,...,...
698,66de4f9c-0aca-11e7-82f6-3863bb44ef7c,4 Ave & 9 St,0,31
1751,cae4f2f4-9642-474c-95cc-6cc07592f120,31 St & Newtown Ave,0,37
691,1877653505745721834,74 St & Woodside Ave,0,19
1754,d7c12bf1-1ec8-496a-beca-90ba76c53241,48 St & Barnett Ave,0,18


In [18]:
all_station_data.sort_values("regular_bikes_available", ascending=False)[["station_id", "name", "regular_bikes_available", "capacity"]]

,station_id,name,regular_bikes_available,capacity
1661,1960020817312746312,Allen St & Hester St,75,117
1927,66dc462a-0aca-11e7-82f6-3863bb44ef7c,W 15 St & 7 Ave,71,79
666,66dc7659-0aca-11e7-82f6-3863bb44ef7c,FDR Drive & E 35 St,66,123
1812,66dbf140-0aca-11e7-82f6-3863bb44ef7c,E 2 St & 2 Ave,63,84
69,66de25bd-0aca-11e7-82f6-3863bb44ef7c,3 St & 3 Ave,63,79
...,...,...,...,...
514,2b33334b-243c-4a16-a6d2-32304c8884f1,Popham Ave & W 174 St,0,20
516,e666a344-1afb-4512-b2e9-2fc5ed71be2b,31 St & Ditmars Blvd,0,24
519,db91c9e0-d80b-4cb4-9cc2-9453ec8fbeac,Grand Concourse & E Mount Eden Ave,0,19
1704,42bf7170-61ab-44a0-aafe-1b89d7271718,Prospect Park SW & 10 Ave,0,30


## What do you want to learn from the real-time Citi Bike data? How could you capture this over time?

---

---

---